# üß¨ Step 3 ‚Äî Descriptor Calculation  
**üìÖ Date:** 21 August 2025  
**üë©‚Äçüî¨ Author:** Mohammed Farzana Begum  
**üìÅ Notebook Path:** notebooks/step3_descriptor_setup.ipynb  

---

## üéØ Objective  
Generate molecular descriptors for curated meroterpenoids using RDKit and Mordred. These descriptors will serve as input features for ML modeling in Step 4.


In [1]:
from rdkit import Chem
print(Chem.MolFromSmiles("CCO"))


In [2]:
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdMolDescriptors
import pandas as pd
import numpy as np


## üì• Load Curated SMILES  
Load canonical SMILES from the cleaned dataset. These molecules have been standardized and deduplicated in Step 2.


In [6]:
import os
print(os.getcwd())


c:\Users\farza\Biotecnika-ML-Screening-of-Plant-Meroterpenoids-for-Anticancer-Activity-Farzana\notebooks


In [2]:
import pandas as pd

df_raw = pd.read_csv("../data/raw_meroterpenoids.csv")
df_raw.head()


compound_id                        smiles
0     CMPD001              CC1=CC(=O)C=CC1O
1     CMPD002           CCC2C1CCC(C2)C(=O)O
2     CMPD003  CC(C)C1=CC=C(C=C1)C(C)C(=O)O

In [4]:
from rdkit import Chem
from rdkit.Chem import SaltRemover
import pandas as pd

df_raw = pd.read_csv("../data/raw_meroterpenoids.csv")

def standardize_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        remover = SaltRemover.SaltRemover()
        mol = remover.StripMol(mol)
        Chem.SanitizeMol(mol)
        return Chem.MolToSmiles(mol, canonical=True)
    except:
        return None

df_raw["canonical_smiles"] = df_raw["smiles"].apply(standardize_smiles)
df_curated = df_raw.dropna(subset=["canonical_smiles"])
df_curated.to_csv("../data/curated_smiles.csv", index=False)
df_curated.head()


compound_id                        smiles           canonical_smiles
0     CMPD001              CC1=CC(=O)C=CC1O           CC1=CC(=O)C=CC1O
1     CMPD002         C1CCC2C(C1)CCC2C(=O)O        O=C(O)C1CCC2CCCCC21
2     CMPD003  CC(C)C1=CC=C(C=C1)C(C)C(=O)O  CC(C)c1ccc(C(C)C(=O)O)cc1

In [5]:
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdMolDescriptors
import pandas as pd
import numpy as np

# Load curated SMILES
df = pd.read_csv("../data/curated_smiles.csv")

# Define descriptor functions
def compute_basic_descriptors(mol):
    return {
        "MolWt": Descriptors.MolWt(mol),
        "LogP": Descriptors.MolLogP(mol),
        "TPSA": rdMolDescriptors.CalcTPSA(mol),
        "NumHDonors": rdMolDescriptors.CalcNumHBD(mol),
        "NumHAcceptors": rdMolDescriptors.CalcNumHBA(mol),
        "NumRotatableBonds": rdMolDescriptors.CalcNumRotatableBonds(mol),
        "RingCount": rdMolDescriptors.CalcNumRings(mol),
    }

def compute_ecfp(mol, n_bits=1024, radius=2):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    vec = np.zeros(n_bits, dtype=int)
    for i in fp.GetOnBits():
        vec[i] = 1
    return vec

# Build descriptor matrix
rows = []
for _, row in df.iterrows():
    mol = Chem.MolFromSmiles(row["canonical_smiles"])
    if mol is None:
        continue
    desc = compute_basic_descriptors(mol)
    fp = compute_ecfp(mol)
    for i, bit in enumerate(fp):
        desc[f"ECFP_{i}"] = bit
    desc["compound_id"] = row["compound_id"]
    rows.append(desc)

# Save matrix
desc_df = pd.DataFrame(rows).set_index("compound_id")
desc_df.to_csv("../data/descriptor_matrix.csv")
desc_df.head()


[20:52:37] DEPRECATION WARNING: please use MorganGenerator
[20:52:37] DEPRECATION WARNING: please use MorganGenerator
[20:52:37] DEPRECATION WARNING: please use MorganGenerator


MolWt    LogP  TPSA  NumHDonors  NumHAcceptors  \
compound_id                                                     
CMPD001      124.139  0.4325  37.3           1              2   
CMPD002      168.236  2.2874  37.3           1              1   
CMPD003      192.258  2.9981  37.3           1              1   

             NumRotatableBonds  RingCount  ECFP_0  ECFP_1  ECFP_2  ...  \
compound_id                                                        ...   
CMPD001                      0          1       0       0       0  ...   
CMPD002                      1          2       0       0       0  ...   
CMPD003                      3          1       0       1       0  ...   

             ECFP_1014  ECFP_1015  ECFP_1016  ECFP_1017  ECFP_1018  ECFP_1019  \
compound_id                                                                     
CMPD001              0          0          0          0          0          1   
CMPD002              0          0          0          0          0          1   
CMPD003              0          0          0          0          0          0   

             ECFP_1020  ECFP_1021  ECFP_1022  ECFP_1023  
compound_id                                              
CMPD001              0          0          0          0  
CMPD002              0          0          0          1  
CMPD003              0          0          0          0  

[3 rows x 1031 columns]

In [1]:
import pandas as pd

# Assuming you have these two lists already
compound_ids = ['CMPD001', 'CMPD002', 'CMPD003']  # Replace with your actual IDs
canonical_smiles = ['CC(C)C1=CC=CC=C1C(=O)O', 'COC1=CC=CC=C1C(=O)O', 'CCC(=O)OC1=CC=CC=C1']  # Replace with your actual SMILES

# Create the DataFrame
curated_df = pd.DataFrame({
    'compound_id': compound_ids,
    'SMILES': canonical_smiles
})

# Save to CSV
curated_df.to_csv('../data/curated_smiles.csv', index=False)
print("‚úÖ curated_smiles.csv saved to data/ folder")


‚úÖ curated_smiles.csv saved to data/ folder
